<a href="https://colab.research.google.com/github/rohandhiman5/Kaggle-challenge-by-cynaptics-club/blob/main/splitting_of_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download spot-the-synthetic

mkdir: cannot create directory ‘/root/.kaggle’: File exists
spot-the-synthetic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
file = 'spot-the-synthetic.zip'
with ZipFile(file, 'r') as zip:
    zip.extractall()

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
import matplotlib.pyplot as plt

In [ ]:
#create ai and real imaage folder in a folder
import os
import shutil
def select_first_two_folders(folder_path):
    folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    selected_folders = folders[0:3:2]
    return selected_folders

source_folder_path = '/content/Data'
destination_folder_path = 'images'

selected_folders = select_first_two_folders(source_folder_path)
if not os.path.exists(destination_folder_path):
    os.makedirs(destination_folder_path)

for folder in selected_folders:
    source = os.path.join(source_folder_path, folder)
    destination = os.path.join(destination_folder_path, folder)
    shutil.copytree(source, destination)
    print(f"Folder '{folder}' copied to '{destination_folder_path}'")


In [ ]:
#create test and validate data folderr
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = '/content/images'
filter_dir = os.path.join(base_dir, 'filter')
test_dir = os.path.join(filter_dir, 'test')
validate_dir = os.path.join(filter_dir, 'validate')
os.makedirs(test_dir, exist_ok=True)
os.makedirs(validate_dir, exist_ok=True)

test_ai_images_dir = os.path.join(test_dir, 'AI_Images')
test_real_images_dir = os.path.join(test_dir, 'Real_Images')
validate_ai_images_dir = os.path.join(validate_dir, 'AI_Images')
validate_real_images_dir = os.path.join(validate_dir, 'Real_Images')

os.makedirs(test_ai_images_dir, exist_ok=True)
os.makedirs(test_real_images_dir, exist_ok=True)
os.makedirs(validate_ai_images_dir, exist_ok=True)
os.makedirs(validate_real_images_dir, exist_ok=True)

AI_Images = os.listdir(os.path.join(base_dir, 'AI_Images'))
Real_Images = os.listdir(os.path.join(base_dir, 'Real_Images'))

test_ai_images, validate_ai_images = train_test_split(AI_Images, train_size=0.8, random_state=42)
test_real_images, validate_real_images = train_test_split(Real_Images, train_size=0.8, random_state=42)

for image in test_ai_images:
    shutil.move(os.path.join(base_dir, 'AI_Images', image), os.path.join(test_ai_images_dir, image))

for image in test_real_images:
    shutil.move(os.path.join(base_dir, 'Real_Images', image), os.path.join(test_real_images_dir, image))

for image in validate_ai_images:
    shutil.move(os.path.join(base_dir, 'AI_Images', image), os.path.join(validate_ai_images_dir, image))

for image in validate_real_images:
    shutil.move(os.path.join(base_dir, 'Real_Images', image), os.path.join(validate_real_images_dir, image))

print("Data split into test and validate sets.")


In [ ]:
base_dir = '/content/images/filter'
train_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validate')

In [ ]:
train_ai_dir = os.path.join(train_dir, 'AI_Images')

train_real_dir = os.path.join(train_dir, 'Real_Images')

validation_ai_dir = os.path.join(validation_dir, 'AI_Images')

validation_real_dir = os.path.join(validation_dir, 'Real_Images')

In [ ]:

train_datagen = ImageDataGenerator(rescale = 1.0/255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))
validation_generator = test_datagen.flow_from_directory(validation_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))



Found 1004 images belonging to 2 classes.
Found 251 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (224,224, 3), include_top = False, weights = 'imagenet')

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:

x = layers.Flatten()(base_model.output)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 50, epochs = 30)

In [ ]:
model.save('model1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import joblib
joblib.dump(model, 'model_filename.pkl')


['model_filename.pkl']

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import os
import numpy as np

In [ ]:
import csv
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input


test_images_folder = '/content/Data/Test_Images'


image_filenames = os.listdir(test_images_folder)


results = []


for filename in image_filenames:

    img_path = os.path.join(test_images_folder, filename)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)


    predictions = model.predict(img_array.reshape(1, 224, 224, 3))


    predicted_class = predictions[0][0]
    predicted_label = "Real" if predicted_class < 0.5 else "AI"


    image_id = int(filename.split('.')[0].split('_')[1])

    results.append({'Id': image_id, 'Label': predicted_label})

results.sort(key=lambda x: x['Id'])

csv_file_path = 'rohan21.csv'


with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Id', 'Label'])
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Predictions saved to '{csv_file_path}'")



In [ ]:
y_pred = model.predict(X_test)
y_pred[:]

In [ ]:
y_classes = [np.argmax(element) for element in y_pred]
y_classes[:]

In [ ]:
labels = ['Real' if label == 1 else 'AI' for label in y_classes]
ids = list(y_classes)
# Print the first few elements of id and label for verification
print("ids:", ids[:5])  # Print the first 5 elements of ids
print("labels:", labels[:5])  # Print the first 5 elements of labels


In [ ]:
print(ids[:])
print(labels[:])

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'id': ids, 'label': labels})